In [ ]:
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
import pandas as pd

In [6]:
train_df = pd.read_csv('train_df.csv')
test_df = pd.read_csv('test_df.csv')
X = train_df.drop('scalar_coupling_constant', axis=1)
X = train_df.drop('molecule_name', axis=1)
Y = train_df['scalar_coupling_constant']

### Boosting

In [12]:
import lightgbm as lgb
import xgboost as xgb 
import catboost as cbt 

N_fold = 5
index = np.arange(len(X))

models = []

def train(model_dict, modelname='lgb'):
    print("-------------------------")
    model = model_dict[modelname]
    print(model)
    model.fit(X[index%N_fold!=i], Y[index%N_fold!=i], 
                eval_set=[(X[index%N_fold==i], Y[index%N_fold==i])], 
                verbose=2, 
                early_stopping_rounds=100
                )
    models.append(model)
    return 

model_dict = {
    'lgb': lgb.LGBMRegressor(objective='regression_l1', n_estimators=200, learning_rate=0.01, max_depth=2,num_leaves=35),
    'xgb': xgb.XGBRegressor(tree_method='hist', objective='reg:absoluteerror', n_estimators=100, max_depth=5, num_leaves=10),
    'cbt': cbt.CatBoostRegressor(objective='MAE', iterations=200, max_depth=5, learning_rate=0.01),
}

for i in range(N_fold):
    train(model_dict, 'lgb')
    train(model_dict, 'xgb')
    train(model_dict, 'cbt')

-------------------------
RandomForestRegressor(random_state=42)


TypeError: BaseForest.fit() got an unexpected keyword argument 'eval_set'

### Simple RandomForest

In [ ]:
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X, Y)

In [ ]:
def log_mae(y_true, y_pred):
    mae = mean_absolute_error(y_true, y_pred)
    log_mae = np.log(mae)
    return log_mae

In [ ]:
X_test = test_df.drop('scalar_coupling_constant', axis=1)
X_test = test_df.drop('molecule_name', axis=1)
Y_test = test_df['scalar_coupling_constant']
y_pred = rf_model.predict(X_test)

print("Score :", log_mae(Y_test, y_pred))